In [1]:
import os
import cv2
import numpy as np
import glob

# Import of Metavision Machine Learning binding
import metavision_sdk_ml
import metavision_sdk_cv
from metavision_sdk_core import EventBbox
from metavision_sdk_core import BaseFrameGenerationAlgorithm
from metavision_core.event_io.py_reader import EventDatReader
from metavision_core.event_io.events_iterator import EventsIterator
from metavision_core.event_io import EventNpyReader

%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [11, 7]

In [20]:
event_file = "17-04-11_15-13-23_122500000_182500000"
event_dat_path = "/home/tkyen/opencv_practice/data_3/Gen1_Automotive/detection_dataset_duration_60s_ratio_1.0/test/{}_td.dat".format(event_file)
event_npy_path = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_evt/{}_td.npy".format(event_file)
gt_label_path = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/{}_bbox.npy".format(event_file)

video_dir = "./video_ev_gt"
os.makedirs(video_dir, exist_ok=True)

event_dat = EventDatReader(event_dat_path)
event_dat.seek_time(0)
ev_height, ev_width = event_dat.get_size()

events_all = np.load(event_npy_path)
labels_all = np.load(gt_label_path)
print(labels_all)
delta_t = 5000

# VideoWriter
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
frame_rate = 10
video_writer = cv2.VideoWriter(os.path.join(video_dir, '{}.mp4'.format(event_file)), fourcc, frame_rate, (ev_width, ev_height))
colors = [[0,0,255], [0,255,0]]
ev_dtype = {'names':['x','y','p','t'], 'formats':['<u2','<u2','<i2','<i8'], 'offsets':[0,2,4,8], 'itemsize':16}

for i in range(int(60e6//delta_t)):    
    evts = events_all[(events_all['t']>=i*delta_t) * (events_all['t']<(i+1)*delta_t)]
    evts_t = evts['t']
    evts_x = evts['x']
    evts_y = evts['y']
    evts_p = evts['p']
    evts = np.concatenate((evts_x[:,None], evts_y[:,None], evts_p[:,None], evts_t[:,None]), axis=1)
    evts_list = evts.tolist()
    evts_list = list(map(tuple, evts_list))
    events = np.empty((0,4), dtype=ev_dtype)
    events = np.append(events, np.asarray(evts_list, dtype=ev_dtype))

    image_all = np.zeros((ev_height, ev_width, 3), dtype=np.uint8)
    BaseFrameGenerationAlgorithm.generate_frame(events, image_all)

    cv2.putText(image_all, '{} ms / {} ms'.format(i*delta_t//1000, (i+1)*delta_t//1000), 
                (ev_width-200, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(image_all, 'GT',
                (10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

    gt_boxes = labels_all[(labels_all['t']>=i*delta_t) * (labels_all['t']<(i+1)*delta_t)]
    for gt_box in gt_boxes:
        t, x, y, w, h, class_id, confidence, track_id, invalid = gt_box
        x, y, w, h = int(x), int(y), int(w), int(h)
        if invalid == False:
            cv2.rectangle(image_all, (x, y), (x+w, y+h), colors[class_id], 1)
    video_writer.write(image_all)

video_writer.release()

[(99999, 40., 144., 52., 33., 0, 1., 202, False)]


In [2]:
event_dat_dir = "/home/tkyen/opencv_practice/data_3/Gen1_Automotive/detection_dataset_duration_60s_ratio_1.0/test"
gt_label_dir = "/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl"
dt_label_dir = "/home/tkyen/opencv_practice/HMNet/experiments/detection/workspace/hmnet_B3_yolox/result/pred_test_10"
video_dir = dt_label_dir + "_video"
os.makedirs(video_dir, exist_ok=True)

event_dat_list = sorted(glob.glob(os.path.join(event_dat_dir, "*.dat")))
gt_label_list = sorted(glob.glob(os.path.join(gt_label_dir, "*.npy")))
dt_label_list = sorted(glob.glob(os.path.join(dt_label_dir, "*.npy")))

In [25]:
for i, (event_dat_path, gt_label_path, dt_label_path) in enumerate(zip(event_dat_list, gt_label_list, dt_label_list)):
    if i > 5:
        break
    event_file = os.path.basename(event_dat_path).replace('_td.dat', '')
    gt_file = os.path.basename(gt_label_path).replace('_bbox.npy', '')
    dt_file = os.path.basename(dt_label_path).replace('_bbox.npy', '')
    assert event_file==gt_file==dt_file

    event_dat = EventDatReader(event_dat_path)
    event_dat.seek_time(0)
    ev_height, ev_width = event_dat.get_size()

    gt_label = EventNpyReader(gt_label_path)
    gt_label.seek_time(0)

    dt_label = EventNpyReader(dt_label_path)
    dt_label.seek_time(0)

    delta_t = 50000

    # VideoWriter
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    frame_rate = 10
    video_writer = cv2.VideoWriter(os.path.join(video_dir, '{}.mp4'.format(event_file)), fourcc, frame_rate, (ev_width*2, ev_height))
    colors = [[0,0,255], [0,255,0]]

    for i in range(int(60e6//delta_t)):
        events = event_dat.load_delta_t(delta_t=delta_t)
        gt_boxes = gt_label.load_delta_t(delta_t=delta_t)
        dt_boxes = dt_label.load_delta_t(delta_t=delta_t)

        image_all = np.zeros((ev_height, ev_width*2, 3), dtype=np.uint8)
        BaseFrameGenerationAlgorithm.generate_frame(events, image_all[:,:ev_width, :])
        image_all[:,ev_width:, :] = image_all[:,:ev_width, :].copy()
        cv2.putText(image_all, '{} ms / {} ms'.format(i*delta_t//1000, (i+1)*delta_t//1000), 
                    (ev_width-200, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image_all, 'GT',
                    (10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image_all, 'DT',
                    (ev_width+10, 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        image_all = cv2.line(image_all, (ev_width,0), (ev_width,ev_height), (0,255,255), 1)

        for gt_box in gt_boxes:
            t, x, y, w, h, class_id, confidence, track_id, invalid = gt_box
            x, y, w, h = int(x), int(y), int(w), int(h)
            if invalid == False:
                cv2.rectangle(image_all, (x, y), (x+w, y+h), colors[class_id], 1)

        for dt_box in dt_boxes:
            t, x, y, w, h, class_id, track_id, confidence = dt_box
            if confidence > 0.4:
                x, y, w, h = int(x), int(y), int(w), int(h)
                cv2.rectangle(image_all, (ev_width+x, y), (ev_width+x+w, y+h), colors[class_id], 1)

        video_writer.write(image_all)
    video_writer.release()
    # break

/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_243500000_bbox.npy Timestamps are not monotonic
/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-04_11-00-13_cut_15_183500000_24350

In [29]:
dt_label = EventNpyReader(dt_label_path)
dt_label.seek_time(0)

In [32]:
while 1:
    dt_boxes = dt_label.load_delta_t(5000)
    if len(dt_boxes) > 0:
        print(dt_boxes)
        break

[(60000, 103.47125, 139.07794, 57.60663 , 33.27533 , 0, 0, 0.9034182 )
 (60000, 264.98108, 122.42304, 39.01892 , 65.67785 , 0, 0, 0.02851164)
 (60000, 262.36133, 130.877  , 40.019714, 48.435944, 0, 0, 0.01577923)]


In [33]:
gt_label = EventNpyReader(gt_label_path)
gt_label.seek_time(0)

In [35]:
while 1:
    gt_boxes = gt_label.load_delta_t(5000)
    if len(gt_boxes) > 0:
        print(gt_boxes)
        break

[(599999, 0., 126., 110., 81., 0, 1., 272, False)]


In [21]:
ev_height, ev_width = event_dat.get_size()
event_dat.seek_time(0)
events = event_dat.load_delta_t(delta_t=delta_t)

In [36]:
ev_height
ev_width

304

In [22]:
record_dat = EventDatReader("/home/tkyen/opencv_practice/data_1/Gen4_Automotive/test_dat/moorea_2019-06-17_test_01_000_1525500000_1585500000_td.dat")
rc_height, rc_width = record_dat.get_size()

record_dat.seek_time(0)
record_events = record_dat.load_delta_t(delta_t=delta_t)


In [23]:
rc_height

720

In [14]:
gt_label = np.load("/home/tkyen/opencv_practice/data_1/Gen1_Automotive/HMNet/test_lbl/17-04-11_15-13-23_122500000_182500000_bbox.npy")
print(gt_label)
gt_ts = np.unique(gt_label['t'])
print(gt_ts)
# for i in range(len(gt_ts)//8):
#     print(gt_ts[i*8:(i+1)*8])
# print(gt_ts[(i+1)*8:])

[(99999, 40., 144., 52., 33., 0, 1., 202, False)]
[99999]
